In [1]:
# define rooms and items

couch = {
    "name": "sofá",
    "type": "furniture",
}

door_a = {
    "name": "porta a",
    "type": "door",
}

door_b = {
    "name": "porta b",
    "type": "door",
}

door_c = {
    "name": "porta c",
    "type": "door",
}

door_d = {
    "name": "porta d",
    "type": "door",
}

key_a = {
    "name": "chave para porta a",
    "type": "key",
    "target": door_a,
}

key_b = {
    "name": "chave para porta b",
    "type": "key",
    "target": door_b,
}

key_c = {
    "name": "chave para porta c",
    "type": "key",
    "target": door_c,
}

key_d = {
    "name": "chave para porta d",
    "type": "key",
    "target": door_d,
}

piano = {
    "name": "piano",
    "type": "furniture",
}

double_bed = {
    "name": "cama de solteiro",
    "type": "furniture",
}

dresser = {
    "name": "armário",
    "type": "furniture",
}

queen_bed = {
    "name": "cama de casal",
    "type": "furniture",
}

dining_table = {
    "name": "mesa de jantar",
    "type": "furniture",
}

game_room = {
    "name": "game room",
    "type": "room",
}

bedroom_1 = {
    "name": "quarto 1",
    "type": "room",
}

bedroom_2 = {
    "name": "quarto 2",
    "type": "room",
}

living_room = {
    "name": "sala de estar",
    "type": "room",
}

outside = {
  "name": "lado de fora"
}

all_rooms = [game_room, bedroom_1, bedroom_2, living_room, outside]

all_doors = [door_a, door_b, door_c, door_d]

# define which items/rooms are related

object_relations = {
    "game room": [couch, piano, door_a],
    "piano": [key_a],
    "porta a": [game_room, bedroom_1],
    "quarto 1": [queen_bed, door_a, door_b, door_c],
    "cama de casal": [key_b],
    "porta b": [bedroom_1, bedroom_2],
    "porta c": [bedroom_1, living_room],
    "quarto 2": [double_bed, dresser, door_b],
    "cama de solteiro": [key_c],
    "armário": [key_d],
    "sala de estar": [dining_table, door_c, door_d],
    "porta d": [outside],
    "lado de fora": [door_d]
}

# define game state. Do not directly change this dict. 
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This 
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside
}

In [2]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")
    
def linebreak1():
    """
    Print a line break
    """
    print("\n")

def start_game():
    """
    Start the game
    """
    print("Você acordou em um sofá numa casa estranha, sem janelas e totalmente desconhecida. Você não se lembra porque você está aqui e o que aconteceu. Você sente que algum perigo está por vir e você deve fugir da casa RÁPIDO!")
    play_room(game_state["current_room"])

def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either 
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print("Parabéns!!! Você escapou!!")
    else:
        print("Você está agora no " + room["name"])
        intended_action = input("O que você quer fazer? Digite 'explorar' or 'examinar'? ").strip()
        if intended_action == "explorar":
            explore_room(room)
            play_room(room)
        elif intended_action == "examinar":
            examine_item(input("O que você quer examinar? ").strip())
        else:
            print("Opção inválida. Digite 'explorar' or 'examinar'. ")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("Você explorou o quarto. Este é o " + room["name"] + ". Você encontrou " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been 
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None
    
    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "Você examinou o " + item_name + ". "
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    output += "Você destrancou a porta com a chave que você tem. "
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "Porta trancada. Você não tem a chave. "
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "Você encontrou " + item_found["name"] + "."
                else:
                    output += "Não há nada de interessante aqui. "
            print(output)
            break

    if(output is None):
        print("O item requisitado não foi encontrado neste quarto. ")
    
    if(next_room and input("Você deseja ir para o próximo quarto? Digite 'sim' or 'não' ").strip() == 'sim'):
        play_room(next_room)
    else:
        play_room(current_room)

In [3]:
game_state = INIT_GAME_STATE.copy()

start_game()

Você acordou em um sofá numa casa estranha, sem janelas e totalmente desconhecida. Você não se lembra porque você está aqui e o que aconteceu. Você sente que algum perigo está por vir e você deve fugir da casa RÁPIDO!
Você está agora no game room
O que você quer fazer? Digite 'explorar' or 'examinar'? explorar
Você explorou o quarto. Este é o game room. Você encontrou sofá, piano, porta a
Você está agora no game room
O que você quer fazer? Digite 'explorar' or 'examinar'? examinar
O que você quer examinar? sofá
Você examinou o sofá. Não há nada de interessante aqui. 
Você está agora no game room
O que você quer fazer? Digite 'explorar' or 'examinar'? examinar
O que você quer examinar? piano
Você examinou o piano. Você encontrou chave para porta a.
Você está agora no game room
O que você quer fazer? Digite 'explorar' or 'examinar'? examinar
O que você quer examinar? porta a
Você examinou o porta a. Você destrancou a porta com a chave que você tem. 
Você deseja ir para o próximo quarto? 